In [1]:
from dotenv import load_dotenv
load_dotenv()
import os

## Transaction Details Module
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("Application Front-End").getOrCreate()


In [2]:
# Load Data to Dataframe from Databse
df1=spark.read.format("jdbc").options(driver="com.mysql.cj.jdbc.Driver",\
                                      user=os.getenv("MYSQL_USER"),\
                                      password=os.getenv("MYSQL_PASS"),\
                                      url="jdbc:mysql://{}:3306/creditcard_capstone".format(os.getenv("MYSQL_HOST")),\
                                      dbtable="CDW_SAPP_BRANCH").load()

df2=spark.read.format("jdbc").options(driver="com.mysql.cj.jdbc.Driver",\
                                      user=os.getenv("MYSQL_USER"),\
                                      password=os.getenv("MYSQL_PASS"),\
                                      url="jdbc:mysql://{}:3306/creditcard_capstone".format(os.getenv("MYSQL_HOST")),\
                                      dbtable="CDW_SAPP_CREDIT_CARD").load()

df3=spark.read.format("jdbc").options(driver="com.mysql.cj.jdbc.Driver",\
                                      user=os.getenv("MYSQL_USER"),\
                                      password=os.getenv("MYSQL_PASS"),\
                                      url="jdbc:mysql://{}:3306/creditcard_capstone".format(os.getenv("MYSQL_HOST")),\
                                      dbtable="CDW_SAPP_CUSTOMER").load()

# Romove Existing Temporary Tables
spark.catalog.dropTempView("Branch")
spark.catalog.dropTempView("Card")
spark.catalog.dropTempView("Customer")

# Creat new Tables
df1.createTempView("Branch")
df2.createTempView("Card")
df3.createTempView("Customer")


In [3]:
#1)  Used to display the transactions made by customers living in a given zip code for a given month and year. Order by day in descending order.
Inputzip=60142
Inputyr=2018
Inputmon=9
spark.sql("SELECT * FROM Card \
          JOIN Customer ON Customer.SSN=Card.CUST_SSN \
          WHERE Customer.CUST_ZIP='{}' and TIMEID Like concat({},lpad({}, 2,'0'),'%')\
          ORDER BY TIMEID DESC".format(Inputzip, Inputyr, Inputmon)).show()

+----------------+--------+---------+-----------+----------------+-----------------+--------------+---------+----------+-----------+---------+----------------+--------------------+---------+----------+-------------+--------+----------+--------------------+--------------------+
|  CREDIT_CARD_NO|  TIMEID| CUST_SSN|BRANCH_CODE|TRANSACTION_TYPE|TRANSACTION_VALUE|TRANSACTION_ID|      SSN|FIRST_NAME|MIDDLE_NAME|LAST_NAME|  Credit_card_no| FULL_STREET_ADDRESS|CUST_CITY|CUST_STATE| CUST_COUNTRY|CUST_ZIP|CUST_PHONE|          CUST_EMAIL|        LAST_UPDATED|
+----------------+--------+---------+-----------+----------------+-----------------+--------------+---------+----------+-----------+---------+----------------+--------------------+---------+----------+-------------+--------+----------+--------------------+--------------------+
|4210653316474645|20180928|123457245|         31|       Education|            37.61|         14278|123457245|    Rodger|      ronny|   Chacon|4210653316474645|   Summ

In [4]:
#2)  Used to display the number and total values of transactions for a given type.
Inputtype='Test'
spark.sql("SELECT COUNT(TRANSACTION_ID) AS Number_of_transactions, SUM(TRANSACTION_VALUE) AS Total_Values FROM Card \
          WHERE TRANSACTION_TYPE='{}' \
          GROUP BY TRANSACTION_TYPE".format(Inputtype)).show()

+----------------------+------------------+
|Number_of_transactions|      Total_Values|
+----------------------+------------------+
|                  6683|341310.37000000005|
+----------------------+------------------+



In [5]:
#3)  Used to display the number and total values of transactions for branches in a given state.
InputState='MN'
spark.sql("SELECT COUNT(TRANSACTION_ID) AS Number_of_Transactions, SUM(TRANSACTION_VALUE) AS Transactions_Total_Values FROM Card \
          JOIN Branch ON Card.BRANCH_CODE=Branch.BRANCH_CODE\
          WHERE BRANCH_STATE='{}' \
          GROUP BY BRANCH_STATE".format(InputState)).show()

+----------------------+-------------------------+
|Number_of_Transactions|Transactions_Total_Values|
+----------------------+-------------------------+
|                  1127|       56153.289999999935|
+----------------------+-------------------------+



In [6]:
## Customer Details Module


In [7]:
#1) Used to check the existing account details of a customer.
InputSSN=123454487

spark.sql("SELECT * FROM Customer \
          WHERE SSN='{}'".format(InputSSN)).show()

#InputCardno=4210653310356919
#spark.sql("SELECT * FROM Customer WHERE Credit_card_no='{}'".format(InputCardno)).show()

+---------+----------+-----------+---------+----------------+--------------------+---------+----------+-------------+--------+----------+-------------------+--------------------+
|      SSN|FIRST_NAME|MIDDLE_NAME|LAST_NAME|  Credit_card_no| FULL_STREET_ADDRESS|CUST_CITY|CUST_STATE| CUST_COUNTRY|CUST_ZIP|CUST_PHONE|         CUST_EMAIL|        LAST_UPDATED|
+---------+----------+-----------+---------+----------------+--------------------+---------+----------+-------------+--------+----------+-------------------+--------------------+
|123454487|    Wilber|   ezequiel|   Dunham|4210653310116272|12th Street East,683|  Huntley|        IL|United States|   60142|  124-3018|WDunham@example.com|2018-04-21T12:49:...|
+---------+----------+-----------+---------+----------------+--------------------+---------+----------+-------------+--------+----------+-------------------+--------------------+



In [11]:
#2) Used to modify the existing account details of a customer.

#from pyspark.sql.functions import when

InputSSN=123454487

#Method 1

#df3n = df3.withColumn("FIRST_NAME", when(df3.SSN == "{}".format(InputSSN),"Harry"))
 
#df3n.filter(df3.SSN =="{}".format(InputSSN)).show(truncate=False)

    
#df3n.write.format("jdbc").mode("overwrite").option("driver","com.mysql.jdbc.Driver")\
#    .option("url", "jdbc:mysql://{}:3306/creditcard_capstone".format(os.getenv("MYSQL_HOST"))) \
#	.option("dbtable", "CDW_SAPP_CUSTOMER") \
#	.option("user", os.getenv("MYSQL_USER")).option("password", os.getenv("MYSQL_PASS")).save()


# Methond 2
import mysql.connector

mydb = mysql.connector.connect(
  host=os.getenv("MYSQL_HOST"),
  user=os.getenv("MYSQL_USER"),
  password=os.getenv("MYSQL_PASS"),
  database="creditcard_capstone"
)

mycursor = mydb.cursor()

sql = "UPDATE CDW_SAPP_CUSTOMER SET FIRST_NAME = %s WHERE SSN = %s"
val=("Yiting", InputSSN)

mycursor.execute(sql, val)


mydb.commit()

print(mycursor.rowcount, "record(s) affected")

1 record(s) affected


In [14]:
#3) Used to generate a monthly bill for a credit card number for a given month and year.
Mon=5
Yr=2018
Cardno=4210653310356919
spark.sql("SELECT SUM(TRANSACTION_VALUE) AS Monthly_Bill FROM Card\
          JOIN Customer ON Card.Credit_card_no=Customer.Credit_card_no\
          WHERE card.Credit_card_no='{}' AND TIMEID Like concat({},lpad({}, 2,'0'),'%')\
          GROUP BY card.Credit_card_no".format(Cardno, Yr, Mon)).show()

+------------+
|Monthly_Bill|
+------------+
|       56.28|
+------------+



In [15]:
#4) Used to display the transactions made by a customer between two dates. Order by year, month, and day in descending order.
#Assume Date2 > Date1

Date1=20180123
Date2=20180325
InputSSN=123454487

spark.sql("SELECT * FROM Card JOIN Customer ON Customer.SSN=Card.CUST_SSN \
          WHERE SSN='{}'AND TIMEID>='{}' AND TIMEID<='{}'\
          ORDER BY TIMEID DESC".format(InputSSN,Date1,Date2)).show()

+----------------+--------+---------+-----------+----------------+-----------------+--------------+---------+----------+-----------+---------+----------------+--------------------+---------+----------+-------------+--------+----------+-------------------+--------------------+
|  CREDIT_CARD_NO|  TIMEID| CUST_SSN|BRANCH_CODE|TRANSACTION_TYPE|TRANSACTION_VALUE|TRANSACTION_ID|      SSN|FIRST_NAME|MIDDLE_NAME|LAST_NAME|  Credit_card_no| FULL_STREET_ADDRESS|CUST_CITY|CUST_STATE| CUST_COUNTRY|CUST_ZIP|CUST_PHONE|         CUST_EMAIL|        LAST_UPDATED|
+----------------+--------+---------+-----------+----------------+-----------------+--------------+---------+----------+-----------+---------+----------------+--------------------+---------+----------+-------------+--------+----------+-------------------+--------------------+
|4210653310116272|20180321|123454487|        105|             Gas|            88.42|         28702|123454487|    Wilber|   ezequiel|   Dunham|4210653310116272|12th Stree